<p>
<font size='10' face='Georgia, Arial'>IIC2115 - Programación como Herramienta para la Ingeniería</font><br>
</p>

# Consultas sobre bases de datos relacionales

Tal como se indicó anteriormente, en esta parte de la materia nos centraremos en la consulta y manipulación de datos con SQL, a través de su MDL.

## Lenguaje de manipulación de datos (DML)
Este subconjunto de SQL permite declarar consultas que se ejecutarán sobre una o más tablas.

### Estructura básica de una consulta
La estructura básica de una consulta en SQL es la siguiente:

> SELECT [DISTINCT] select-list <br>
> FROM from-list <br>
> WHERE qualification <br>

Cada consulta debe tener una cláusula **SELECT**, que especifica las columnas que deben conservarse en el resultado, y una cláusula **FROM**, que especifica un producto cruzado de tablas. La cláusula opcional **WHERE** especifica las condiciones de selección en las tablas mencionadas en la cláusula FROM. Consideremos a continuación un ejemplo sencillo, encontrar los nombres y edades de todos los estudiantes:

> SELECT DISTINCT E.nombre, E.edad <br>
> FROM Estudiantes E

La respuesta a la consulta anterior es un conjunto de filas, cada una de las cuales es un par (nombre, edad). Si dos o más estudiantes tienen el mismo par (nombre, edad) la respuesta contendrá sólo un par de estos. Si omitimos el modificador **DISTINCT**, obtendríamos una copia de la fila, para cada estudiante con el mismo nombre y edad. Esto se conoce como un *multiconjunto*, que a diferencia de un conjunto tradicional, puede contener valores repetidos. 

La siguiente consulta agrega una restricción simple a la consulta anterior:

> SELECT * <br>
> FROM Estudiantes E <br>
> WHERE E.promedio > 5.0

Existen dos diferencia principales entre esta consulta y la anterior. Primero, dado que queremos obtener todos los datos de un alumno, no es necesario especificar cada columna, ya que basta con ingresar un \*. Segundo, dentro de los registros de la tabla Estudiantes, sólo retornaremos aquellos que tienen un promedio superior a 5.0.

Consideremos ahora la sintaxis de una consulta SQL básica con más detalle:

* La from-list en la cláusula FROM es una lista de nombres de tabla. Un nombre de tabla puede ser seguido por una variable de rango; una variable de rango es particularmente útil cuando el mismo nombre de la tabla aparece más de una vez en la from-list.
* La select-list es una lista de nombres de columnas de tablas nombradas en la from-list.
* Los argumentos en la cláusula WHERE son una combinación booleana (es decir, una expresión utilizando los conectivos lógicas AND, OR y NOT) de condiciones con forma *expresión op expresión*, donde *op* es uno de los siguientes operadores (<, <=, =, <>,> =,>). Una expresión es un nombre de columna, un constante o una expresión aritmética o de string.

### Consultas entre tablas (JOIN)
A continuación veremos consultas de ejemplo, donde es necesario comparar o combinar elementos de distintas tablas, lo que se conoce como un **join**. El primer ejemplo present una consulta que encuentra a los alumnos de postgrado de 24 o más años.

> SELECT E.nombre  <br>
> FROM Estudiantes E, Estudiantes_Postgrado P  <br>
> WHERE E.id = P.id AND E.edad >= 24

Como se aprecia, sólo se deben comparar las ocurrencias de **id**, ya que esta columna aparece en ambas tablas. Por el contrario, **edad**, al estar sólo en la tabla **Estudiantes**, no necesita comparación.

Una situación más compleja se puede ver en la siguiente consulta, donde buscamos los cursos en que está inscrito cada estudiante:

> SELECT E.nombre, C.nombre  <br>
> FROM Estudiantes E, Cursos C, Inscrito I  <br>
> WHERE E.id = I.id_estudiante AND C.id = I.id_curso

Esta consulta contiene un join de tres tablas, ya que es necesario pasar por la tabla **Inscrito** para relacionar a los estudiates con sus cursos.

La última consulta básica que revisaremos involucra el ordenamiento de los resultados de manera transparente para el usuario:

> SELECT E.nombre  <br>
> FROM Estudiantes E, Estudiantes_Postgrado P  <br>
> WHERE E.id = P.id AND edad >= 24  <br>
> ORDER BY edad ASC

Como se puede ver, la consulta es una extensión de la primera consulta básica revisada, donde ahora los resultados se ordenarán de menor a mayor, en base a la edad de los estudiantes.

### Matching de patrones en strings
El siguiente tipo de consulta que revisaremos son bastante comunes, ya que involucran la búsqueda sobre strings, con patrones que no necesariamente calzan completamente con la información almacenada. Por ejemplo, si actualmente quisieramos buscar todos los alumnos cuyos nombre comiencen con "MA", sería necesario realizar una cantidad enorme de consultas, considerando todas las posibles combinaciones. Afortunamente, SQL entrega un mecanismo simple para solucionar esta situación:

> SELECT \*  <br>
> FROM Estudiantes E  <br>
> WHERE E.nombre LIKE 'MA%'

Existen multiples patrones de strings que pueden verificarse, no solo patrones de inicio. Más información sobre esto puede encontrarse [acá](https://www.w3schools.com/sql/sql_like.asp).

### Manipulación y comparación de conjuntos
SQL proporciona también constructos de manipulación y comparación de conjuntos que extienden las consultas básicas presentadas anteriormente. Como la respuesta a una consulta es un conjunto de múltiples filas, es
natural considerar el uso de operaciones tales como unión, intersección y diferencia. SQL admite estas operaciones bajo los nombres UNION, INTERSECT y EXCEPT (o MINUS). SQL también proporciona otras operaciones de conjunto: IN (para verificar si un elemento está en un conjunto dado), op ANY, op ALL (para comparar un valor con los elementos de un conjunto dado, usando el operador de comparación op), y EXISTS (para verificar si un conjunto es vacío). IN y EXISTS pueden ser prefijados por NOT, con la modificación obvia a su significado. Como ejemplos, considere al siguiente consulta, que busca todos los identificadores de los alumnos con promedio superior a 5.0 **o** que estén en el curso con identificador IIC2115:

> SELECT E.id <br>
> FROM Estudiantes E  <br>
> WHERE E.promedio > 5.0 <br>
> UNION <br>
> SELECT I.id_estudiante <br>
> FROM Inscritos I <br>
> WHERE I.id_curso = 'IIC2115'

Si quisieramos encontrar los alumnos que tienen promedio superior a 5.0 **y** están en IIC2115, basta con intercambiar UNION por INTERSECT. Más aun, si quisieramos encontrar todos los alumnos con promedio superior a 5.0 y que **no** están en IIC2115, bastan con cambiar UNION (o INTERSECT) por EXCEPT. 

Los operadores de comparación de conjuntos los analizaremos en el contexto del próximo tipo de consultas.


### Consultas anidadas
Una de las características más potentes de SQL son las consultas anidadas. Una consulta anidada es una consulta que tiene otra consulta incrustada dentro de ella; la consulta incrustada se llama subconsulta (_subquery_). La consulta incrustrada puede ser, por supuesto, una consulta anidada en sí mismoa; por lo tanto, son posibles consultas que tienen multiples niveles de anidación. La necesidad de este tipo de consultas nace cuando escribiendo una consulta se necesita expresar una condición, que debe calcularse en el momento haciendo referencia también a una tabla. La consulta utilizada para calcular esta tabla subsidiaria es una subconsulta y aparece como parte de la consulta principal. Una subconsulta típicamente aparece dentro de la cláusula WHERE.

Para comenzar, consideremos la siguiente consulta, que retorna el nombre de todos los alumnos que **no** se encuentren inscritos en el curso IIC2115

> SELECT E.nombre <br>
> FROM Estudiantes E  <br>
> WHERE E.id NOT IN (SELECT I.id_estudiante <br>
> &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;FROM Inscritos I <br>
> &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;WHERE I.id_curso = 'IIC2115')

También es posible hacer que la subconsulta dependa de la consulta principal. Para verlo, consideremos la siguiente consulta, cuyo resultado es el mismo que el de la consulta anterior:

> SELECT E.nombre <br>
> FROM Estudiantes E  <br>
> WHERE EXISTS (SELECT * <br>
> &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;FROM Inscritos I <br>
> &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;WHERE I.id_curso = 'IIC2115' AND I.id_estudiante = E.id)

Esta consulta también ilustra el uso del símbolo $*$ en situaciones donde todo lo que queremos hacer es verificar que exista una cierta fila, y realmente no queremos recuperar ninguna columna de la fila. Este es uno de los pocos usos de $*$ en la cláusula SELECT que se considera un buen estilo de programación.

La comparación de conjuntos sigue un esquema bastante similar. Por ejemplo, si buscamos los alumnos cuyo promedio es superior al de todos los alumnos de nombre José, podemos utilizar la siguiente consulta anidada:

> SELECT E.nombre <br>
> FROM Estudiantes E  <br>
> WHERE E.promedio > ALL (SELECT E2.promedio <br>
> &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;FROM Estudiantes E2 <br>
> &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;WHERE E2.nombre = 'José')

### Agregación
Además de simplemente recuperar datos, a menudo queremos realizar algún cálculo o agregación de estos. A continuación, revisaremos una poderosa clase de constructos para calcular valores agregados como MIN y SUM. SQL soporta cinco operaciones agregadas, que pueden aplicarse en cualquier columna, digamos A, de una relación:

* COUNT ([DISTINCT] A): La cantidad de valores (unicos) en la columna A.
* SUM ([DISTINCT] A): La suma de todos los valores (unicos) en la columna A.
* AVG ([DISTINCT] A): El promedio de todos los valores (unicos) en la columna A.
* MAX (A): El máximo valor en la columna A.
* MIN (A): El mínimo valor en la columna A.

Veamos a continuación un ejemplo simple:

> SELECT AVG (E.edad)  <br>
> FROM Estudiantes E

La consulta anterior permite calcular de manera eficiente el promedio de las edades de todos los alumnos registrados. Si deseamos filtrar un poco más los resultados, podemos realizar la siguiente consulta :

> SELECT AVG (E.edad)  <br>
> FROM Estudiantes E  <br>
> WHERE E.promedio > 5.0

Cabe destacar que todos los elementos vistos anteriormente para la cláusula WHERE, pueden ser usados en conjunto con las funciones de agregación.


### Agrupación
Hasta ahora, hemos aplicado operaciones agregadas a todas las filas de una relación. A menudo, queremos aplicar operaciones agregadas a ciertos grupos de filas en una relación, donde el número de grupos depende de la instancia de relación (es decir, no se conoce de antemano). Por ejemplo, si queremos calcular el promedio de edad de los estudiantes, agrupados por sus promedios, deberíamos realizar algo por el estilo:

> SELECT AVG (E.edad)  <br>
> FROM Estudiantes E  <br>
> WHERE E.promedio = i

con i = 1,.., 7 (asumiendo notas enteras, como se indica en el esquema de la relación). Claramente, escribir 7 veces la misma consulta es tedioso, pero más importante aún, no siempre se conoce a priori el número de categorías por los que se quiere agrupar.

Para facilitar este tipo de problemas, SQL incluye una extensión a las consultas básicas en la forma de la cláusula **GROUP BY** (y el filtro opcional **HAVING**). Usando esta nueva funcionalidad, podemos escribir la consulta anterior de la siguiente manera:

> SELECT AVG(E.edad) <br>
> FROM Estudiantes E  <br>
> GROUP BY E.promedio <br>
> HAVING E.promedio >= 5

### Valores nulos
Hasta ahora, hemos asumido que los valores de columna en una fila son siempre conocidos. En la práctica, esto no es siempre cierto, ya que algunos de estos valores pueden ser desconocidos. Por ejemplo, cuando un alumo se inscribe en un curso, es natural que aún no tenga una nota asignada. Dado que la definición para la tabla Inscrito tiene una columna nota, ¿qué deberíamos insertar en esa columna? Lo que se necesita aquí es un valor especial que denote algo desconocido.

SQL proporciona un valor de columna especial llamado **NULL** para usar en tales situaciones. Se utiliza este valor cuando el contenido de la columna sea desconocido o inaplicable. Para verificar la existencia de un valor **NULL**, SQL proporciona el operador de comparación **IS NULL**.

## Consultas en SQL a través de Python
Para ejemplificar el uso de Python para realizar consultas, continuaremos con el ejemplo utilizado en la parte anterior. Para facilitar las cosas, repetiremos el código para crear y llenar la base de datos.

In [ ]:
import sqlite3

connection = sqlite3.connect('example.db')   
cursor = connection.cursor()                    
cursor.execute("CREATE TABLE countries(name TEXT, continent TEXT, population INTEGER, language TEXT)")
countries = [('Andorra','Europe',77281, 'Catalan'),
             ('China','Asia', 1403500365, 'Mandarin'),
             ('Jamaica', 'America', 2881355, 'English'),
             ('Chile', 'America', 18006407, 'Spanish'),
             ('Australia', 'Oceania', 24680100, 'English')]
cursor.executemany('INSERT INTO countries VALUES (?,?,?,?)', countries)
connection.commit() 
connection.close()

Si queremos verificar que la inserción fue realizada correctamente, debemos ejecutar una consulta sobre la misma base de datos.

In [ ]:
connection = sqlite3.connect('example.db')
cursor = connection.cursor()
cursor.execute('SELECT * FROM countries')

#obtenemos sólo una fila, si queremos todas, debemos usar fetchall (no cambia el resultado en este caso)
print(cursor.fetchone())
connection.close()

Como se indicó en la parte anterior de la materia, otra manera de revisar los contenidos de la base de datos, es utilizando una herramienta especializada para esto. Dentro de la gran variedad existente, una opción simple y liviana es [SQLiteStudio](sqlitestudio.pl), que también es multiplataforma (Windows, Linux, Mac).

### Parametrización de consultas

La manera más directa y sencilla de introducir información variable en una consulta, es utilizando *strings*. Por ejemplo, si permitimos a un usuario indicar el nombre del país sobre el cuál se buscará información, es posible construir una consulta parametrizada de la siguiente manera:

In [ ]:
connection = sqlite3.connect('example.db')
cursor = connection.cursor()
country = "'China'"
cursor.execute("SELECT * FROM countries WHERE name = %s" % country)
print(cursor.fetchone())

A pesar de que la consulta anterior entregó el resultado esperado, el usar este tipo de estrategia para parametrizar consultas es un error muy común en la gran mayoría de los programas que interactuan con bases de datos. Esto puede generar grandes problemas de seguridad, debido a un ataque conocido como *SQL Injection*, que consiste en introducir, como dato para una consulta, consultas completas de SQL que pueden causar grandes pérdidas en la base de datos. Veamos el siguiente caso como ejemplo, donde eliminaremos completamente la tabla *countries* utilizando esta técnica:

In [ ]:
#connection = sqlite3.connect('example.db')
#cursor = connection.cursor()
country = "'China'; DROP TABLE countries;"
# a diferencia de execute, el método executescript permite ejecutar más de una consulta de 
# manera secuencial (esto abre la puerta a la ejecución de código indeseado).
cursor.executescript("SELECT * FROM countries WHERE name = %s" % country)

#verificamos la existencia de la tabla
cursor.execute('PRAGMA table_info([countries])')
print(cursor.fetchall())

Para evitar situaciones de este tipo, es fundamental utilizar una técnica segura, como la **sustitución de parámetros**. En el caso de SQLite en Python, se debe poner el signo **?** en una consulta, cada vez que se desee sustituir posteriormente con un valor. Posteriormente, basta con introducir como una tupla los valores deseados, como segundo parámetro del método *execute*.

In [ ]:
cursor.execute("CREATE TABLE countries(name TEXT, continent TEXT, population INTEGER, language TEXT)")
countries = [('Andorra','Europe',77281, 'Catalan'),
             ('China','Asia', 1403500365, 'Mandarin'),
             ('Jamaica', 'America', 2881355, 'English'),
             ('Chile', 'America', 18006407, 'Spanish'),
             ('Australia', 'Oceania', 24680100, 'English')]
cursor.executemany('INSERT INTO countries VALUES (?,?,?,?)', countries)

country = ('Andorra',)
cursor.execute('SELECT * FROM countries WHERE name = ?', country)
print(cursor.fetchall())

A diferencia del uso de *strings*, la sustitución de parámetros, al hacer explícita la existencia de información externa, permite realizar verificaciones de seguridad que impiden la ejecución de código malicioso. De esta manera, si intentamos realizar nuevamente un ataque de *SQL Injection*, el resultado será distinto:

In [ ]:
country = ("'China'; DROP TABLE countries;",)
cursor.execute("SELECT * FROM countries WHERE name = ?", country)
cursor.execute('PRAGMA table_info([countries])')
print(cursor.fetchall())

Finalmente, con el fin de hacer más limpia (y formateable) la impresión de los resultados obtenidos por una consulta, es posible utilizar el cursor resultante de una consulta como un iterador:

In [ ]:
for country in cursor.execute('SELECT * FROM countries ORDER BY name'):
    print(country)
connection.close()